In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
from os import listdir
from datasets import Dataset, load_from_disk
from decord import VideoReader, cpu
from transformers import VideoMAEFeatureExtractor, VideoMAEModel

In [2]:
clip_list = listdir("D:\BundesligaKaggle\DflBundesligaDataShootout\clips")

In [3]:
len(clip_list)

200

In [4]:
def sample_frame_indices(clip_len, start_idx, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = start_idx + converted_len
    indices = np.linspace(start_idx, (end_idx-frame_sample_rate), num=clip_len).astype(np.int64)
    return indices

feature_extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

In [5]:
df = pd.DataFrame(columns = ['vid_init_frame', 'clips'])

In [12]:
i = 0
r = 0

for clip in clip_list:
    
    with open('D:\BundesligaKaggle\DflBundesligaDataShootout\clips/' + clip_list[0], 'rb') as f:
        vr = VideoReader(f, num_threads=2, ctx=cpu(0))
        
        for ini in range(750//30):
            if ini < 24:
                indix = sample_frame_indices(16,ini*30,2,750)
            else:
                indix = sample_frame_indices(16,ini*30-1,2,750)
            print(indix)
            batch = vr.get_batch(indix).asnumpy()
            inputs = feature_extractor(list(batch), return_tensors='np')
            inputs1D = inputs.flatten() 
            print(inputs.shape)
            df.at[r, 'vid_init_frame'] = clip.split('.')[0] + '_' + str(ini*30)
            df.at[r, 'clips'] = inputs1D
            r += 1
    
    if i <= 0:
        clip_dataset = Dataset.from_pandas(df)
        clip_dataset.save_to_disk("D:\BundesligaKaggle\DflBundesligaDataShootout\clip_mate")
        print(r)
    else:
        break
    i += 1

[ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28 30]


AttributeError: 

In [ ]:
reloaded_dataset = load_from_disk("clip_mate")

In [ ]:
len(reloaded_dataset)